# Plan-and-Execute
---

### What is Plan-and-Execute?
The Plan-and-Execute framework is a strategy for retrieval-augmented generation (RAG) that divides complex reasoning tasks into two distinct phases: planning and execution. While traditional ReAct agents think one step at a time, plan-and-execute emphasizes explicit, long-term planning.

- **Planning Phase**: The model generates a high-level plan or structured outline that serves as a roadmap for solving the task. This phase ensures that the execution is systematic and adheres to the task's requirements.

- **Execution Phase**: Based on the generated plan, the model retrieves relevant information and executes the outlined steps to provide a detailed and coherent response.

This separation aims to address limitations in RAG systems that attempt to perform reasoning and generation in a single step, often leading to logical errors or inefficiency in handling complex tasks.

### Key Advantages
- **Improved Task Decomposition**: By explicitly separating planning and execution, the framework enables better handling of complex, multi-step reasoning tasks, ensuring systematic progress towards the solution.

- **Higher Accuracy and Coherence**: The planning phase acts as a guide, reducing the chances of errors and improving the logical coherence of the responses generated during execution.

**Reference**
- [ReAct paper](https://arxiv.org/abs/2210.03629)
- [Plan-and-Solve paper](https://arxiv.org/abs/2305.04091)
- [Baby-AGI project](https://github.com/yoheinakajima/babyagi)  


In [17]:
from dotenv import load_dotenv
import os
import json

load_dotenv("../../.env")

from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient



azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY", "") if len(os.getenv("AZURE_OPENAI_API_KEY", "")) > 0 else None
azure_openai_chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002")
openai_api_version = os.getenv("OPENAI_API_VERSION", "2024-06-01")

<br>

## 🧪 Step 1. Test and Construct each module
---

Before building the entire the graph pipeline, we will test and construct each module separately.

### Web Search Tool

Web search tool is used to enhance the context.

In [18]:
# LANGUAGE = "English"
# LOCALE = "en-US"
LANGUAGE = "Korean"
LOCALE = "ko-KR"

language_prompt = f" Answer in {LANGUAGE}."

In [19]:
from azure_genai_utils.tools import BingSearch

WEB_SEARCH_FORMAT_OUTPUT = False

web_search_tool = BingSearch(
    max_results=2,
    locale=LOCALE,
    include_news=False,
    include_entity=False,
    format_output=WEB_SEARCH_FORMAT_OUTPUT,
)
tools = [web_search_tool]

In [20]:
question = "Microsoft AutoGen"
results = web_search_tool.invoke({"query": question})
print(results[0])

{'kind': 'web', 'title': 'AutoGen - Microsoft Research', 'snippet': '<b>AutoGen</b> is an open-source programming framework for building AI agents and facilitating cooperation among multiple agents to solve tasks. It supports asynchronous messaging, modular and extensible components, observability and debugging, scalability and distribution, and cross-language interoperability.', 'content': 'Open-Source Framework for Agentic AI aka.ms/autogen(opens in new tab) autogen@microsoft.com AutoGen is an open-source programming framework for building AI agents and facilitating cooperation among multiple agents to solve tasks. AutoGen aims to provide an easy-to-use and flexible framework for accelerating development and research on agentic AI. Over the past year, our work on AutoGen has highlighted the transformative potential of agentic AI in addressing real-world challenges through agents and multi-agent applications. Building on this progress, we are excited to announce AutoGen v0.4—a signifi

In [21]:
autogen_aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=azure_openai_endpoint,
    model = azure_openai_chat_deployment_name,
    api_version=openai_api_version,
    api_key=azure_openai_key
)

<br>


## 🧪 Step 1. Define the Agentic Architecture
- Before building the agentic pipeline, we need to design the message, topic, agent and message routing logic. 
- You should define the terminate condition for the pipeline.

### Message, Topic, Agent Definition

```markdown
```python

# Message Definition
ChatMessage
termination = TextMentionTermination("TERMINATE")

# Agent Definition
planner_agent = AssistantAgent(gpt-4o)
web_search_agent  = AssistantAgent(gpt-4o)
data_analyst  = AssistantAgent(gpt-4o)


# Message Routing Definition
RoundRobinGroupChat
```
```

### Define Agents


This system uses three specialized agents:

![selector-group-chat](../../images/selector-group-chat.svg)

Planning Agent: The strategic coordinator that breaks down complex tasks into manageable subtasks.
Web Search Agent: An information retrieval specialist that interfaces with the search_web_tool.
Data Analyst Agent: An agent specialist in performing calculations equipped with

#### Workflow
The task is received by the SelectorGroupChat which, based on agent descriptions, selects the most appropriate agent to handle the initial task (typically the Planning Agent).

The Planning Agent analyzes the task and breaks it down into subtasks, assigning each to the most appropriate agent using the format: <agent> : <task>

Based on the conversation context and agent descriptions, the SelectorGroupChat manager dynamically selects the next agent to handle their assigned subtask.

The Web Search Agent performs searches one at a time, storing results in the shared conversation history.

The Data Analyst processes the gathered information using available calculation tools when selected.

The workflow continues with agents being dynamically selected until either:

The Planning Agent determines all subtasks are complete and sends “TERMINATE”

An alternative termination condition is met (e.g., a maximum number of messages)

In [26]:
LOCALE = "en-US"
WEB_SEARCH_FORMAT_OUTPUT = False

def web_search_tool(query: str) -> str:
    
    web_search_client = BingSearch(
        max_results=3,
        locale=LOCALE,
        include_news=False,
        include_entity=False,
        format_output=WEB_SEARCH_FORMAT_OUTPUT,
    )
    results = web_search_client.invoke({"query": query})
    return results[0]
    
def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [27]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=autogen_aoai_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[web_search_tool],
    model_client=autogen_aoai_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=autogen_aoai_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

<br>

## 🧪 Step 3. Execute the SelectorGroupChat
---

### Execute the SelectorGroupChat

In [28]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=autogen_aoai_client,
    termination_condition=termination,
)

In [29]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_9YrLzX7qIO071AhXqeEEplq0', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='web_search_tool'), FunctionCall(id='call_Ci48M8wuM014o2vhWp3oTP8L', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season"}', name='web_search_tool'), FunctionCall(id='call_qnuuKYsFdFN6p1ctDbtf48pj', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season"}', name='web_search_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='{\'kind\': \'web\', \'title\': \'2006-07 Miami Heat Player Stats - Regular Season - LandOfBasketball.com\', \'snippet\': \'Statis per game recorded by the <b>2006-2007</b> <b>Heat</b> <b>players</b> in the Regular <b>Season</b>, inlc

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=94), content=[FunctionCall(id='call_9YrLzX7qIO071AhXqeEEplq0', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='web_search_tool'), FunctionCall(id='call_Ci48M8wuM014o2vhWp3oTP8L', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season"}', name='web_search_tool'), FunctionCall(id='call_qnuuKYsFdFN6p1ctDbtf48pj', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season"}', name='web_search_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, content=[FunctionExecutionResult(content='{\'kind